In [3]:
print("oK")

oK


In [1]:
pip install ctransformers==0.2.5

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install sentence-transformers==2.2.2

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pinecone-client

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install langchain==0.0.225

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain-0.0.225-py3-none-any.whl (1.2 MB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.1
    Uninstalling langchain-0.2.1:
      Successfully uninstalled langchain-0.2.1
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install flask

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install pypdf

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install -U langchain-community

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 2.1 MB 2.4 MB/s eta 0:00:01
  Using cached langchain-0.2.1-py3-none-any.whl (973 kB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.225
    Uninstalling langchain-0.0.225:
      Successfully uninstalled langchain-0.0.225
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import Pinecone
from pinecone import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

/Users/gouravmukherjee/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/gouravmukherjee/Library/Python/3.9/lib/python/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
PINECONE_API_KEY="ef6d21a0-5c08-4de8-aab8-e28778d54cb4"
# PINECONE_API_ENV=""

In [21]:
#Extract Data from PDF
def load_pdf(data):
    loader= DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documets=loader.load()
    return documets


In [23]:
# extracted_data
extracted_data = load_pdf("data/")

In [24]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [119]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [5]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [6]:
embeddings = download_hugging_face_embeddings()

In [7]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [5]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [36]:
#Initializing the Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
index= pc.Index("medical-chatbot")

embedding= [embeddings.embed_query(t.page_content) for t in text_chunks]
#Creating Embeddings for Each of The Text Chunks & storing
vectors=[{
    "id": f"vec{i+1}",
    "values": embedding[i],
    "metadata" : {"text": str(text_chunks[i].page_content)}
}
 for i in range(0,1000)#len(text_chunk)
]
namespace = "medical_chatbot"


In [ ]:
index.upsert(
    vectors = vectors,
    namespace= namespace
)
print("vector Uploaded")

In [80]:
# Define the query
query = "What are Acne"

query_embd=embeddings.embed_query(query)

In [ ]:
query_embd

In [81]:

# Perform a query using the query method
results = index.query(
    namespace="medical_chatbot",
    vector=query_embd,  # Replace with your query vector
    top_k=3,
    include_values=True,
    include_metadata=True,
    # filter={"genre": {"$eq": "medical"}}
)
# Print the search results
print("Results:", results["matches"][0]['metadata']['text'])



Results: Acidosis seeRespiratory acidosis; Renal
tubular acidosis; Metabolic acidosis
Acne
Definition
Acne is a common skin disease characterized by
pimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.
Description
Acne vulgaris, the medical term for common acne, is


In [8]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [9]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [10]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':256,
                          'temperature':0.8})

In [ ]:
llm

In [113]:
%pip install --upgrade --quiet  langchain-pinecone langchain-openai langchain


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [11]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore(embedding=embeddings, index_name="medical-chatbot")

In [12]:
docsearch = PineconeVectorStore.from_existing_index("medical-chatbot",embeddings)

In [13]:
docsearch

In [ ]:
docsearch = PineconeVectorStore.from_documents([t for t in text_chunks], embeddings, index_name="medical-chatbot")

In [14]:
query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE", metadata={'page': 130.0, 'source': 'data/Medical_book.pdf'}), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additive

In [3]:
try:
    print("Loading the model...")
    llm = CTransformers(
        model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
        model_type="llama",
        config={'max_new_tokens': 256, 'temperature': 0.8}
    )
    print("Model loaded successfully!")
    
except Exception as e:
    print(f"Failed to load the model: {e}")
    

Loading the model...
Model loaded successfully!


In [17]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 1}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
    )



In [18]:
result=qa.invoke({"query": "what is Acne?"})

In [19]:
print("Response : ", result["result"])

Response :  Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.


In [13]:
#while True:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])

/Users/gouravmukherjee/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


KeyboardInterrupt: 

In [ ]:
import pinecone

# Initialize Pinecone
pinecone.init(api_key="ef6d21a0-5c08-4de8-aab8-e28778d54cb4")

# Set the name of your existing Pinecone index
index_name = "medical-chatbot"

# Retrieve the existing index by its name
index = pinecone.Index(index_name)

In [104]:
import pinecone
from langchain.vectorstores import Pinecone
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY)

index_name="medical-chatbot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [103]:
#Initializing the Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY,)
index= pc.Index("medical-chatbot")

TypeError: __init__() got an unexpected keyword argument 'api_key'

In [99]:
docsearch = Pinecone.from_existing_index(embedding= embeddings,index_name="medical-chatbot",namespace='medical_chatbot')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/gouravmukherjee/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/qr/g0spfbl53255v6cn3h0gml940000gn/T/ipykernel_9227/382213007.py", line 1, in <module>
    docsearch = Pinecone.from_existing_index(embedding= embeddings,index_name="medical-chatbot",namespace='medical_chatbot')
  File "/Users/gouravmukherjee/Library/Python/3.9/lib/python/site-packages/langchain/vectorstores/pinecone.py", line 354, in from_existing_index
TypeError: __init__() missing 1 required positional argument: 'host'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/gouravmukherjee/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/gouravmukherjee/Library

In [93]:
from langchain.vectorstores import Pinecone
# # Define your retriever using Pinecone
docsearch = Pinecone.from_existing_index("medical-chatbot", embeddings)
# query = "What are Allergies"

# docs=docsearch.similarity_search(query, k=3)

# print("Result", docs)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/gouravmukherjee/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/qr/g0spfbl53255v6cn3h0gml940000gn/T/ipykernel_9227/1837222940.py", line 3, in <module>
    docsearch = Pinecone.from_existing_index("medical-chatbot", embeddings)
  File "/Users/gouravmukherjee/Library/Python/3.9/lib/python/site-packages/langchain/vectorstores/pinecone.py", line 354, in from_existing_index
TypeError: __init__() missing 1 required positional argument: 'host'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/gouravmukherjee/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/gouravmukherjee/Library/Python/3.9/lib/python/site-packages/IPython/cor

In [6]:
# Initialize Pinecone
import pinecone

# PINECONE_API_KEY = "your_pinecone_api_key"
PINECONE_API_KEY="ef6d21a0-5c08-4de8-aab8-e28778d54cb4"
# Initialize Pinecone client
pinecone_client = pinecone.Pinecone(api_key=PINECONE_API_KEY, environment='us-east-1')  # Adjust environment as needed

# Set the name of your existing Pinecone index
index_name = "medical-chatbot"

# Retrieve the existing index by its name
index = pinecone_client.Index(index_name)

In [108]:
# Create Embeddings and Store them in Pinecone

# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.pinecone import Pinecone as LangChainPinecone

# Load your embeddings model
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Assuming you have a variable `text_chunks` with the data to be embedded
texts = [t.page_content for t in text_chunks]


# Store embeddings in Pinecone
docsearch = LangChainPinecone.from_texts(texts, embeddings, index_name=index_name, pinecone_client=pinecone_client)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/gouravmukherjee/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/qr/g0spfbl53255v6cn3h0gml940000gn/T/ipykernel_9227/1618943751.py", line 14, in <module>
    docsearch = LangChainPinecone.from_texts(texts, embeddings, index_name=index_name, pinecone_client=pinecone_client)
  File "/Users/gouravmukherjee/Library/Python/3.9/lib/python/site-packages/langchain/vectorstores/pinecone.py", line 296, in from_texts
  File "/Users/gouravmukherjee/Library/Python/3.9/lib/python/site-packages/pinecone/deprecation_warnings.py", line 51, in list_indexes
    raise AttributeError(_build_class_migration_message('list_indexes', example))
AttributeError: list_indexes is no longer a top-level attribute of the pinecone package.

To use list_indexes, please create a client instance and call the method there instead.

Example:

    fro